# Recurrent Neural network in numpy step by step

Recurrent Neural Networks (RNN) are one of the powerful tool which can be used to convert sequence data into insight. In this notebook, I am building recurrent network with numpy. 

In [1]:
import numpy as np;

In [25]:
def softmax_func(yhat):
    """
    Input vector y of n*1
    """
    #Shifting max to 0 for numerical stability
    t = np.exp(yhat-np.max(yhat))
    return t/np.sum(t,axis=0)

The following is a simple rnn cell.

In [28]:
def cell_rnn(a_prev, x_curr, weight_dict):
    """
    Input : a_prev : last layer activation function
            x_curr : current input at time t
    Output:  a_curr, y_hat_curr
    """
    W_ax = weight_dict["W_ax"]
    b_a =  weight_dict["b_a"]
    W_aa = weight_dict["W_aa"]
    W_ya = weight_dict["W_ya"]
    b_y =  weight_dict["b_y"]
    a_curr = np.tanh((np.dot(W_ax,x_curr)+np.dot(W_aa,a_prev) + b_a))
    y_hat_curr = softmax_func(np.dot(W_ya,a_curr) + b_y)
    # store backward propagation
    cache = (a_curr, a_prev, x_curr, weight_dict)
    return a_curr,y_hat_curr,cache

### Forward Pass

RNN can be viewed as unrolling above cell k times if sequence is of length k. The output a_curr will become a_prev of the next step and this process will continue k times.
The input sequence $x = (x^{\langle 1 \rangle}, x^{\langle 2 \rangle}, ..., x^{\langle T_x \rangle})$  is carried over $T_x$ time steps. The network outputs $y = (y^{\langle 1 \rangle}, y^{\langle 2 \rangle}, ..., y^{\langle T_x \rangle})$.

We need to store the return value in cache:

* next activation at t
* previous activation at t
* dataX at time t
*  parameter at time t

<img src="input_rnn.PNG" style="width:700px;height:300px;">

In [41]:
def forward_rnn(x,a0,weight_dict):
    """
    Input : 
    x  : sequence data n_x * k * T_x matrix (n_x is number of feature, T_x is time and k is data count)
    a0 : activation at zeroth time. Dimension : n_a * k #
    weight_dict: Parameters dictionary
    weight_dict : All parameters 
    
    Return:
    Cache : It will serve as a storage to the backprop. Value to be stored:
            1. input x 
            2. Activation for every step t for every point 
    y_hat : Dimension : n_x * k * n 
    a_all : Activation of all hidden state
    """
    cache = [] #To hold values during forward prop for backprop 
    no_pass = np.shape(x)[1]
    #Iniitalize yhat and a_0
    n_x = x.shape[0] #no of features
    k = x.shape[1] # number of points
    T_x = x.shape[2] # no of time points
    n_a = a0.shape[0] # activation size x
    n_y = weight_dict["W_ya"].shape[0] #final output size
    y_hat_all = np.zeros((n_y,k,T_x))
    a_all  = np.zeros((n_a,k,T_x))
    #Initial value of a_cur which is a<t>
    a_cur = a0
    for count_t in range(T_x):
        current_x = x[:,:,count_t]
        a_cur, yhat, cache_return = cell_rnn(a_cur,current_x,weight_dict)
        y_hat_all[:,:,count_t] = yhat
        a_all[:,:,count_t] = a_cur
        cache.append(cache_return)
    caches = (cache, x)
    return  a_all,y_hat_all,caches

### Backpropogation Pass

 The cool name for the backprogation algorithm for RNN is called as "backpropogation through time". If you unroll the RNN in time the gradient need to be propogated in time! We will start will the loss whihc is sum of the all the loss till time t. Gradients of the cost with respect to  a_t at every time-step  t helps the gradient backpropagate to the previous RNN-cell.

In [80]:
def backward_rnnCell(da_t_1, cache):
    """
    Input:
        da_prev_t: The gradient at time t+1 if t is the current time
        cache : collection of stored (a<t>,a<t-1>,x<t>,weight_dict)
    Output:
    gradient_t{
        da_t:
        dx_t:
        dW_aa:
        db_a:
        dW_ax:
        }
    """
    a_cur, a_prev, x_t, parameter = cache 
    W_ax = parameter['W_ax']
    W_aa = parameter['W_aa']
    b_a = parameter['b_a']
    dtanh = (1-np.tanh(W_ax.dot(x_t)+W_aa.dot(a_prev)+b_a)**2)*(da_t_1)
    dx_t =  W_ax.T.dot(dtanh)
    da_t = W_aa.T.dot(dtanh)
    dW_aa = dtanh.dot(a_prev.T)
    dW_ax = dtanh.dot(x_t.T)
    db_a = np.sum(dtanh,axis=1,keepdims=True)
    gradient_t ={
        "da_t_1": da_t,
        "dx_t": dx_t,
        "dW_aa": W_aa,
        "db_a":db_a,
        "dW_ax":dW_ax
        }
    return gradient_t;

Coming to the tricky part of backprop. Let da hold are all derivatives of the hidden states over the full sequence for the previous training epoch. When propagating the da_prev of t to the da_next of t-1, we need to add the corresponding da which would be its value in the previous epoch.
            <img src="cool_rnn.png" style="width:700px;height:500px;">

In [99]:
def backward_pass(da, caches):
    """
    Input:
    da : Upstream gradient dimension(n_x, m,T_x )//Calculated using loss 
    caches : tuple of caches (cache, x)
    
    """
    
    (cache, x)  = caches
    (a_curr, a_prev, x_curr, weight_dict)= cache[0]
    #Initialize the final back variables
    ##
    n_x,m,T_x = x.shape
    n_a = a_curr.shape[0]
    dWaa = np.zeros((n_a,n_a)) 
    dWax = np.zeros((n_a,n_x))
    dba = np.zeros((n_a,1))
    dx = np.zeros((n_x,m,T_x))
    da_prev_t = np.zeros((n_a,m)) 
    for t in reversed(range(T_x)):
        total_back = da_prev_t + da[:,:,t]
        gradient_t = backward_rnnCell(total_back,cache=cache[t])
        da_prev_t,dx_t,dWaa_t, dba_t, dWax_t = gradient_t["da_t_1"], gradient_t["dx_t"], gradient_t["dW_aa"],gradient_t["db_a"], gradient_t["dW_ax"]
        dWaa+= dWaa_t
        dWax+= dWax_t
        dba+= dba_t
        dx[:,:,t] = dx_t 
    da0 = da_prev_t
    final_gradient = {"dx":dx,"dWaa":dWaa,"dWax":dWax,"dba":dba,"da0":da0}
    return final_gradient